<div style="float:right; padding-top: 15px; padding-right: 15px">
    <div>
        <a href="https://whiteboxml.com">
            <img src="https://whiteboxml.com/static/img/logo/black_bg_white.svg" width="250">
        </a>
    </div>
</div>

## 0. python imports & setup

for learning purposes, libraries will be imported inside its corresponding usage section...

## 1. data loading

In [1]:
import pandas as pd
import numpy as np

* diamonds: labeled data we can use for training and testing
* diamonds_predict: diamonds to predict its price and upload result to Kaggle

In [2]:
diamonds = pd.read_csv('../data/diamonds_train.csv')
diamonds_predict = pd.read_csv('../data/diamonds_predict.csv')

## 2. features modification

In [3]:
# Creating new variables for diamonds

In [4]:
def set_value(row_number, assigned_value): 
    return assigned_value[row_number] 
  
# Create the dictionary 
color_dictionary ={'D' : 1, 'E' : 1, 'F' : 1, 'G' :0, 'H' :0, 'I' :0, 'J':0 } 
  
# Add a new column named 'color_simplifyed' 
diamonds['color_simplifyed'] = diamonds['color'].apply(set_value, args =(color_dictionary, ))
diamonds_predict['color_simplifyed'] = diamonds_predict['color'].apply(set_value, args =(color_dictionary, ))

In [5]:
def set_value(row_number, assigned_value): 
    return assigned_value[row_number] 
  
# Create the dictionary 
cut_dictionary ={'Ideal' : 5, 'Premium' : 4, 'Very Good' : 3, 'Good' :2, 'Fair' :1 } 
  
# Add a new column named 'cut_numeric' 
diamonds['cut_numeric'] = diamonds['cut'].apply(set_value, args =(cut_dictionary, )) 
diamonds_predict['cut_numeric'] = diamonds_predict['cut'].apply(set_value, args =(cut_dictionary, ))

In [6]:
# Add a new column named 'cut_and_carat' 
diamonds['cut_and_carat'] = diamonds['cut_numeric']*diamonds['carat']
diamonds_predict['cut_and_carat'] = diamonds_predict['cut_numeric']*diamonds_predict['carat']

In [7]:
# Add a new column named 'x minus y squared' 
diamonds['x_minus_y_squared'] = (diamonds['x']-diamonds['y'])**2
diamonds_predict['x_minus_y_squared'] = (diamonds_predict['x']-diamonds_predict['y'])**2

In [8]:
# Create a new column called based on the value of another column
diamonds['roundness'] = np.where(diamonds.x_minus_y_squared <= 0.025, 1, 0)
diamonds_predict['roundness'] = np.where(diamonds_predict.x_minus_y_squared <= 0.025, 1, 0)

In [9]:
# Add a new column named 'xyz' 
diamonds['xyz'] = (diamonds['x']*diamonds['y']*diamonds['z'])
diamonds_predict['xyz'] = (diamonds_predict['x']*diamonds_predict['y']*diamonds_predict['z'])

In [10]:
# Add a new column named 'carat_volume_cut' 
diamonds['carat_volume_cut'] = diamonds['cut_and_carat']/diamonds['xyz']
diamonds_predict['carat_volume_cut'] = diamonds_predict['cut_and_carat']/diamonds_predict['xyz']

In [11]:
diamonds_predict.head().T

,0,1,2,3,4
id,0,1,2,3,4
carat,0.79,1.2,1.57,0.9,0.5
cut,Very Good,Ideal,Premium,Very Good,Very Good
color,F,J,H,F,F
clarity,SI1,VS1,SI1,SI1,VS1
depth,62.7,61,62.2,63.8,62.9
table,60,57,61,54,58
x,5.82,6.81,7.38,6.09,5.05
y,5.89,6.89,7.32,6.13,5.09
z,3.67,4.18,4.57,3.9,3.19


In [12]:
diamonds.shape

(40455, 17)

In [13]:
diamonds_predict.shape

(13485, 17)

as you can see, there are both categorical and numerical columns...

## 2. eda

this section is up to you! this guided lesson is about a machine learning pipeline...

## 3. ml preprocessing

in this section I will teach how to use scikit-learn's Pipiline and ColumnTransformer, one of the best practices for composing preprocessing and modeling in a single and elegand class... pay attention as it is hard to understand...

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

* https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
* https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer
* https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
* https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

let's identify numerical and categorical features...

In [15]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   carat              40455 non-null  float64
 1   cut                40455 non-null  object 
 2   color              40455 non-null  object 
 3   clarity            40455 non-null  object 
 4   depth              40455 non-null  float64
 5   table              40455 non-null  float64
 6   price              40455 non-null  int64  
 7   x                  40455 non-null  float64
 8   y                  40455 non-null  float64
 9   z                  40455 non-null  float64
 10  color_simplifyed   40455 non-null  int64  
 11  cut_numeric        40455 non-null  int64  
 12  cut_and_carat      40455 non-null  float64
 13  x_minus_y_squared  40455 non-null  float64
 14  roundness          40455 non-null  int64  
 15  xyz                40455 non-null  float64
 16  carat_volume_cut   404

In [16]:
NUM_FEATS = ['carat', 'depth', 'table', 'x', 'y', 'z','color_simplifyed','cut_numeric','cut_and_carat','x_minus_y_squared','roundness']
CAT_FEATS = ['cut', 'color', 'clarity']
FEATS = NUM_FEATS + CAT_FEATS
TARGET = 'price'

let's define a preprocessing transformer for numerical columns...

In [17]:
numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), 
                ('scaler', StandardScaler())])

let's define a preprocessing transformer for categorical columns...

In [18]:
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))])

let's join these transformers using a `ColumnTransformer`:

In [19]:
preprocessor =ColumnTransformer(transformers=[('num', numeric_transformer, NUM_FEATS),
                                ('cat', categorical_transformer, CAT_FEATS)])

inspecting the full preprocessor:

In [20]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['carat', 'depth', 'table', 'x', 'y', 'z',
                                  'color_simplifyed', 'cut_numeric',
                                  'cut_and_carat', 'x_minus_y_squared',
                                  'roundness']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['cut',

how does this preprocessing looks like?

In [21]:

type(preprocessor)

sklearn.compose._column_transformer.ColumnTransformer

at least in this case, it is at the cost of interpretability of transformed DataFrame...

In [22]:
pd.DataFrame(data=preprocessor.fit_transform(diamonds)).head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0.867006,0.452019,0.247981,0.978807,0.921985,1.022657,-0.967096,0.085178,0.897154,-0.005195,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-1.004557,0.871099,-0.199745,-1.226738,-1.179816,-1.129259,-0.967096,-0.809387,-1.039390,-0.005250,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-0.184434,2.617265,-1.095198,-0.097286,-0.176882,0.161891,-0.967096,-2.598516,-1.164167,-0.004689,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,-0.815298,1.429872,-0.647472,-0.933258,-0.883296,-0.770607,1.034023,-1.703951,-1.109265,-0.005195,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.467458,-0.875068,0.695707,0.729794,0.677793,0.592274,-0.967096,0.979742,1.026923,-0.005195,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


## 4. train a simple model

first, lets train a simple model using holdout, train - test split...

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
diamonds_train, diamonds_test = train_test_split(diamonds)

In [25]:
print(diamonds_train.shape)
print(diamonds_test.shape)

(30341, 17)
(10114, 17)


let's choose a model from scikit-learn cheatsheet: https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

## list of models

In [26]:
"""""
listb_df = pd.read_csv('Listb.csv')
listb=listb_df.values.tolist()
print(listb)



SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-26-7d931c5ae3e1>, line 5)

In [27]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.isotonic import IsotonicRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import StackingRegressor




In [28]:
model= Pipeline(steps=[('preprocessor', preprocessor),
                       ('regressor',RandomForestRegressor() )])

In [26]:
model.fit(diamonds_train[FEATS], diamonds_train[TARGET]);

## 5. check model performance on test and train data

In [27]:
from sklearn.metrics import mean_squared_error

In [28]:
y_test = model.predict(diamonds_test[FEATS]).clip(325,18500)
y_train = model.predict(diamonds_train[FEATS]).clip(325,18500)

In [29]:
print(f"test error: {mean_squared_error(y_pred=y_test, y_true=diamonds_test[TARGET], squared=False)}")
print(f"train error: {mean_squared_error(y_pred=y_train, y_true=diamonds_train[TARGET], squared=False)}")

test error: 547.8354684792722
train error: 211.8985923504008


In [35]:
test_error=mean_squared_error(y_pred=y_test, y_true=diamonds_test[TARGET], squared=False)
train_error=mean_squared_error(y_pred=y_train, y_true=diamonds_train[TARGET], squared=False)
# track model
mod = input('Algorithm used:')
b=(train_error,test_error,mod)
listb.append(b)
print (listb)
listb_df=pd.DataFrame(listb, columns=["train_error","test_error","model"])
listb_df.to_csv("~/Ironhack/ih_datamadpt0420_project_m3/notebooks/Listb.csv", index=False)

Algorithm used: RandomForestRegressor


NameError: name 'listb' is not defined

In [36]:
display(listb_df)

NameError: name 'listb_df' is not defined

In [85]:
listb_df.to_csv("~/Ironhack/ih_datamadpt0420_project_m3/notebooks/Listb.csv", index=False)

In [26]:
display(pd.DataFrame(listb, columns=["train_error","test_error","model"]))

,train_error,test_error,model
0,212.991025,552.468056,RandomForestRegressor


## 6. check model performance using cross validation

In [30]:
from sklearn.model_selection import cross_val_score

In [31]:
scores = cross_val_score(model, 
                         diamonds[FEATS], 
                         diamonds[TARGET], 
                         scoring='neg_root_mean_squared_error', 
                         cv=5, n_jobs=-1)

In [32]:
import numpy as np
np.mean(-scores)

556.6232857963771

## 7. optimize model using grid search

In [33]:
from sklearn.model_selection import RandomizedSearchCV

In [34]:
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'regressor__n_estimators': [256, 512, 1024],
    'regressor__max_depth': [8,16,32],
}

grid_search = RandomizedSearchCV(model, 
                                 param_grid, 
                                 cv=5, 
                                 verbose=10, 
                                 scoring='neg_root_mean_squared_error', 
                                 n_jobs=-1,
                                 n_iter=18)

grid_search.fit(diamonds[FEATS], diamonds[TARGET])

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   43.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  4.0min
/home/davidgozalo/miniconda3/envs/ironhack_env/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 12.6min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed: 15.4min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 20.7min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed: 24.3min
[Parallel(n_jobs=-1)]: Done  85 out of  90 | elapsed: 31.1min remaining:  1.8min
[Parallel(n_jobs=-1)]: Done  90 out

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               ['carat',
                                                                                'depth',
                                                                                'table',
                                                                                'x',
              

In [35]:
grid_search.best_params_

{'regressor__n_estimators': 512,
 'regressor__max_depth': 16,
 'preprocessor__num__imputer__strategy': 'mean'}

In [36]:
grid_search.best_score_

-553.1082453108859

## 8. prepare submission

In [37]:
y_pred = grid_search.predict(diamonds_predict[FEATS])

In [38]:
submission_df = pd.DataFrame({'id': diamonds_predict['id'], 'price': y_pred})

In [39]:
submission_df.head()

,id,price
0,0,3129.197543
1,1,5459.247599
2,2,9469.062420
3,3,4216.489380
4,4,1693.732503


In [40]:
submission_df.describe()

,id,price
count,13485.000000,13485.000000
mean,6742.000000,3951.831342
std,3892.928525,3949.504448
min,0.000000,371.018866
25%,3371.000000,950.169233
50%,6742.000000,2452.088300
75%,10113.000000,5306.569583
max,13484.000000,18194.383967


In [41]:
submission_df.price.clip(325, 18500, inplace=True)

In [42]:
submission_df.to_csv('diamonds III', index=False)

## 9. let's try more models...

In [29]:
model= Pipeline(steps=[('preprocessor', preprocessor),
                       ('regressor',RandomForestRegressor(n_estimators=512, max_depth=16) )])

In [30]:
model.fit(diamonds[FEATS], diamonds[TARGET]);

In [31]:
y_pred=model.predict(diamonds_predict[FEATS]).clip(275,18500)

In [32]:
submission_df = pd.DataFrame({'id': diamonds_predict['id'], 'price': y_pred})

In [33]:
submission_df.to_csv('diamonds XI', index=False)

<div style="padding-top: 25px; float: right">
    <div>    
        <i>&nbsp;&nbsp;© Copyright by</i>
    </div>
    <div>
        <a href="https://whiteboxml.com">
            <img src="https://whiteboxml.com/static/img/logo/black_bg_white.svg" width="125">
        </a>
    </div>
</div>